### Meditation analysis

In [ ]:
"""
Computes the cond. mutual information between power a network feature
and the stimulus for all sessions.
"""
import sys

sys.path.insert(1, "/home/vinicius/storage1/projects/GrayData-Analysis")
import argparse
import os

import xarray as xr
from frites.dataset import DatasetEphy
from frites.estimator import GCMIEstimator
from frites.workflow import WfMi
from tqdm import tqdm

from config import sessions
from GDa.graphics import plot
from GDa.util import average_stages, create_stages_time_grid

In [ ]:
SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = plot.set_plot_config()

In [ ]:
# Create directory to save figures
if not os.path.exists("img/n6.5"):
    os.makedirs("img/n6.5")

In [ ]:
metric = "degree"
avg = 0

#### Get root path

In [ ]:
_ROOT = os.path.expanduser("~/storage1/projects/GrayData-Analysis")

#### Iterate over all sessions and concatenate power

In [ ]:
sxx = []
for s_id in tqdm(sessions):
    # Uses power for trial type 1
    # behavioral response 1 aligned at cue
    _FILE_NAME = "power_tt_1_br_1_at_cue.nc"
    path_pow = os.path.join(_ROOT, f"Results/lucy/{s_id}/session01", _FILE_NAME)
    power = xr.load_dataarray(path_pow)
    # Averages power for each period (baseline, cue, delay, match) if needed
    out = average_stages(power, avg)

    sxx += [out.isel(roi=[r]) for r in range(len(out["roi"]))]
    del power

#### Iterate over all sessions and concatenate coherece

In [ ]:
coh = []
stim = []
roi = []
for s_id in tqdm(sessions):
    _FILE_NAME = f"coh_{metric}_thr_1_at_cue.nc"
    path_metric = os.path.join(_ROOT, f"Results/lucy/{s_id}/session01", _FILE_NAME)
    # Load network feature
    feature = xr.load_dataarray(path_metric)
    # Average if needed
    out = average_stages(feature, avg)
    # Convert to format required by the MI workflow
    coh += [out.isel(roi=[r]) for r in range(len(out["roi"]))]
    del feature

In [ ]:
# Convert to DatasetEphy
dt = DatasetEphy(coh, y=sxx, z=stim, nb_min_suj=10, times="times", roi="roi")

#### MI Workflow

In [ ]:
# Convert to DatasetEphy
dt = DatasetEphy(coh, y=sxx, z=stim, nb_min_suj=10, times="times", roi="roi")

mi_type = "ccd"
inference = "rfx"
kernel = None
estimator = GCMIEstimator(
    mi_type=mi_type,
    relative=False,
    copnorm=True,
    biascorrect=False,
    demeaned=False,
    tensor=True,
    gpu=False,
    verbose=None,
)
wf = WfMi(mi_type, inference, verbose=True, kernel=kernel, estimator=estimator)

kw = dict(n_jobs=20, n_perm=100)
cluster_th = None  # {float, None, 'tfce'}

mi, pvalues = wf.fit(dt, mcp="cluster", cluster_th=cluster_th, **kw)

In [ ]:
print(sxx[0].shape)
print(coh[0].shape)

In [ ]:
len(stim[0])

In [ ]:
len(roi)